In [ ]:
# Anamorphic Encryption - ElGamal Scheme
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from char_int_elgamal import encode_msg_to_int, decode_msg_to_int
import secrets
import math
from itertools import zip_longest

<h1> ElGamal: Baby step - Giant step </h1>

In [ ]:
# ======== HELPER FUNCTIONS ======== #


# Pseudo-random function
def F(sk_a, IV, st, q):
    prg_input = st.to_bytes(16, 'little')  # AES block size = 16 bytes
    iv_bytes = IV.to_bytes(16, 'little')
    
    aes = AES.new(sk_a, AES.MODE_CBC, iv=iv_bytes)
    encrypted = aes.encrypt(prg_input)
    
    r_prime = (int.from_bytes(encrypted, 'little') % (q - 1)) + 1
    return r_prime

def get_generator(p):
    while True:
        h = secrets.randbelow(p-3) + 2
        g = pow(h, 2, p)
        if g != 1:
            return g


# ======== KEY GENERATION ======== #


# Standard public/private + double key generation
def create_keys(p, q, g):
    sk_a = get_random_bytes(16) # double key
    sk = secrets.randbelow(q) # secret key
    pk = pow(g, sk, p) # public key
    return pk, sk, sk_a


# ======== ANAMORPHIC ENCRYPTION ======== #


def aEncrypt(p, q, g, m, m_a, pk, sk_a, IV, st):
    # Compute pseudo-random part
    r_prime = F(sk_a, IV, st, q)
    r = (r_prime + m_a) % q

    # Calculate ElGamal ciphertext
    pk_r = pow(pk, r, p)
    ct0 = (m * pk_r) % p
    ct1 = pow(g, r, p)
    ct = (ct0, ct1)

    # Increment state for next message
    st += 1

    return ct, st


def baby_step_giant_step(p, g, h, bound):
    if h == 1:
        return 0  # g^0 == 1
    
    # limit bound to at least 1
    if bound <= 1:
        return None

    # n = ceil(sqrt(q))
    n = math.isqrt(bound)
    if n * n < bound:
        n += 1

    # Baby steps: store g^j -> j for j in [0, n-1]
    baby = {}
    cur = 1
    for j in range(n):
        # If collision, keep the smallest j
        if cur not in baby:
            baby[cur] = j
        cur = (cur * g) % p

    # Compute g^n and its modular inverse modulo p
    g_n = pow(g, n, p)
    # modular inverse of g_n modulo p
    g_n_inv = pow(g_n, p - 2, p)

    # giant steps: i from 0..ceil(bound/n)-1
    gamma = h
    max_i = (bound + n - 1) // n  # ceiling(bound / n)
    for i in range(max_i):
        if gamma in baby:
            j = baby[gamma]
            x = i * n + j
            if x < bound:
                return x
            else:
                return None
        gamma = (gamma * g_n_inv) % p

    return None

# Anamorphic decryption
def aDecrypt(p, g, sk_a, ct, IV, st, q, bound):
    r_prime = F(sk_a, IV, st, q)

    # compute inverse of g^r_prime
    g_rprime = pow(g, r_prime, p)
    g_rprime_inv = pow(g_rprime, p - 2, p)
    g_m_a = (ct[1] * g_rprime_inv) % p

    m_a = baby_step_giant_step(p, g, g_m_a, bound)
    return m_a

# Standard decryption
def Decrypt(p, ct, sk):
    s = pow(ct[1], sk, p) # s = g^r^x = y^r
    s_inverse = pow(s, -1, p)
    cover_msg = (ct[0] * s_inverse) % p
    return cover_msg


# ===== Initialization ===== #

p = 25283138329189278652587895589109525736072750946542698825287111445816073512149787631506175333955884685211183346377467560941062660497423325529940869143458703 
q = 12641569164594639326293947794554762868036375473271349412643555722908036756074893815753087666977942342605591673188733780470531330248711662764970434571729351
g = get_generator(p)

pk, sk, sk_a = create_keys(p, q, g)

st = 0
IV = 0

bound = 2**36

In [ ]:
# ===== Encryption ===== #
m = 13000
m_a = 200000005
print(f'm = {m}, m_a = {m_a}')

ct, st_new = aEncrypt(p, q, g, m, m_a, pk, sk_a, IV, st)
print(f"Ciphertext: {ct}")


# ======== Decryption ======== #
cover_m = Decrypt(p, ct, sk)
secret_m = aDecrypt(p, g, sk_a, ct, IV, st, q, bound)
print(f'cover_m = {cover_m}, secret_m = {secret_m}')

In [73]:
def encrypt_m(m: str, m_a: str):
    int_array = [encode_msg_to_int(word) for word in m.split()]
    a_int_array = [encode_msg_to_int(word_a) for word_a in m_a.split()]

    pairs = zip_longest(int_array, a_int_array, fillvalue=0)

    cts = []
    st_local = st
    for m_int, m_a_int in pairs:
        ct, st_local = aEncrypt(p, q, g, m_int, m_a_int, pk, sk_a, IV, st_local)
        cts.append(ct)

    return cts, st_local

def decrypt_and_decode(cts, st_start):
    decoded_cover = ""
    decoded_anam = ""
    st_local = st_start

    for ct in cts:
        m = Decrypt(p, ct, sk)
        m_a = aDecrypt(p, g, sk_a, ct, IV, st_local, q, bound)
        decoded_cover += decode_msg_to_int(m) + " "
        decoded_anam += (decode_msg_to_int(m_a) if m_a is not None else "[?]") + " "
        st_local += 1

    return decoded_cover.strip(), decoded_anam.strip()

# ====== Test ====== #
m = "Hi, how are you doing James?"
m_a = "We attack at dawn, my friend" # Only words <= 6 char-length work

# Encode to ciphertext
cts, st_after = encrypt_m(m, m_a)
print("Ciphertext: ", cts)

# Decode to text
decoded_m, decoded_m_a = decrypt_and_decode(cts, st)
print(f'cover_m: {decoded_m} \nsecret_m: {decoded_m_a}')

Ciphertext:  [(4319412768647108297125022400138766908808955433264181762654848229934387700143670202067411280493526865310777346505561596935529816490812704720337221163026982, 24593054705352255624998955886081538713520065383011171210764188342135555365017230103374176441859126831976137989149272056002321581927150966175143045109428051), (19337392467089686003872511895667668570111404910682407438952626563395962082534597252724346682014704297317161616789661216076953514652471761016254832641695343, 3699636245314863754258033399898374300880283907029821125848433428332499887206064986100871083500822675823494723669993335943368286208344865023300095428159946), (14337867020991038512946008714579018726079048076664345429210736143704757788223902618043537272214854442431644624216517839781000171279355187787384617909268059, 10424730637775264723287862534866460509250050841975263051806213752643820148992221227591431990895981408814494670946795868115580452746062804602483950961107680), (150539732919746536865273683563870188573

### To implement:
* ElGamal ✅
* Baby step / giant step ✅
* Implement static strong prime ✅
* Encrypt messages instead of integers ✅
* Write down progress + knowledge in Overleaf ✅
* Read paper 🛑
* Implement Paillier 🛑
* Implement password protection + panic password 🛑
* Final UI + ai chatbot 🛑